first notebook lamda = 4 and second is lambda = 2
this is 1

# Import dependencies

In [ ]:
from tensorflow import keras
import tensorflow.keras.backend as K
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
from PIL import Image
import warnings
import os
import random
import datetime
from tensorflow.python.ops.gen_array_ops import deep_copy

from losses import IgnoreCaseWeightedLoss
from data_generator import DataGenerator
from metrics import accuracy
from train_val_epoch import ignore_case_train_epoch, ignore_case_validation_epoch
from metrics import plot_confusion_matrix_for_ignore_case, meanf1_iou_for_ignore_case
from metrics import plot_feature_space

%matplotlib inline

In [ ]:
print('TensorFlow {}; Keras {}'.format(tf.__version__, keras.__version__))
print(tf.test.gpu_device_name())

In [ ]:
BASE_PATH = "... path to source dataset"
TAR_BASE_PATH = "... path to target dataset"
test_ratio = 0.1

files = os.listdir(BASE_PATH)
tar_files = os.listdir(TAR_BASE_PATH)
files = [BASE_PATH + f for f in files]
tar_files = [TAR_BASE_PATH + f for f in tar_files]
print("###SRC FILES###")
print(BASE_PATH)
print(len(files))
print("###TAR FILES###")
print(TAR_BASE_PATH)
print(len(tar_files))

In [ ]:
random.seed(10)
random.shuffle(files)
random.shuffle(tar_files)
test_size = int(len(files) * test_ratio)

test_files = files[:len(tar_files)]
non_test_files = files[len(tar_files):]

val_size = int(len(non_test_files) * test_ratio)
val_files = non_test_files[:val_size]
train_files = non_test_files[val_size:]

print("Train size:", len(train_files))
print("Validation size:", len(val_files))
print("Test size:", len(test_files))
print(train_files[0])

In [ ]:
maximum, minimum = 14.733826, -49.551544
BATCH_SIZE = 8

train_gen = DataGenerator(image_paths=train_files, batch_size=BATCH_SIZE,  augment=True, 
                          shuffle=True, normalize=True, maximum=maximum, minimum=minimum)
val_gen = DataGenerator(image_paths=val_files, batch_size=BATCH_SIZE, augment=False, shuffle=False,
                        normalize=True, maximum=maximum, minimum=minimum)
test_gen = DataGenerator(image_paths=test_files, batch_size=BATCH_SIZE, augment=False, shuffle=True,
                         normalize=True, maximum=maximum, minimum=minimum)

tar_gen = DataGenerator(image_paths=tar_files, batch_size=BATCH_SIZE, augment=False, shuffle=True,
                         normalize=True, maximum=maximum, minimum=minimum)

# Loss

In [ ]:
other = 36107543
corn = 551043
cotton = 106433
rice = 23218346
total = other + corn + cotton + rice 

# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
weight_for_other = (1 / other) * (total / 2.0)
weight_for_corn = (1 / corn) * (total / 2.0)
weight_for_cotton = (1 / cotton) * (total / 2.0)
weight_for_rice = (1 / rice) * (total / 2.0)

class_weight = {0: weight_for_other, 1: weight_for_corn, 2: weight_for_cotton, 3: weight_for_rice}

print('Weight for class 0: {:.2f}'.format(weight_for_other))
print('Weight for class 1: {:.2f}'.format(weight_for_corn))
print('Weight for class 2: {:.2f}'.format(weight_for_cotton))
print('Weight for class 3: {:.2f}'.format(weight_for_rice))
print('Weight for class 4: {:.2f}'.format(0.0))

weights = [weight_for_other, weight_for_corn, weight_for_cotton, weight_for_rice]

In [ ]:
weights = [weight_for_other, weight_for_corn*1.5, weight_for_cotton*1.5, weight_for_rice]

# Create domain adaptive UNet

In [ ]:
import models

In [ ]:
NUM_CLASSES = 4

d_model, model = models.unet_2d((256, 256, 12), [16, 32, 32, 64], n_labels=NUM_CLASSES,
                      stack_num_down=2, stack_num_up=1,
                      activation='GELU', output_activation='Softmax', 
                      batch_norm=True, pool='max', unpool='nearest', name='unet',
                      is_domain_adaptation=True, da_type='conv2d', da_kernels=[32, 16])

In [ ]:
model.summary()

In [ ]:
d_model.summary()

In [ ]:
weighted_crossentropy = IgnoreCaseWeightedLoss(keras.losses.categorical_crossentropy, weights, 4)
binary_crossentropy = keras.losses.binary_crossentropy
binary_accuracy = keras.metrics.BinaryAccuracy()

lr_schedule1 = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=10000,
    decay_rate=0.9)
lr_schedule2 = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=10000,
    decay_rate=0.9)
optimizer1 = keras.optimizers.Adam(learning_rate=lr_schedule1)
optimizer2 = keras.optimizers.Adam(learning_rate=lr_schedule2)

# Train

In [ ]:
epochs = 100
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [ ]:
# segmentation_loss_mean = keras.metrics.Mean()
# segmentation_accuracy_mean = keras.metrics.Mean()
# domain_loss_mean = keras.metrics.Mean()
# domain_accuracy_mean = keras.metrics.Mean()

for epoch in range(epochs):
    # train model in one epoch
    (segmentation_loss,
    segmentation_accuracy,
    domain_loss,
    domain_accuracy) = ignore_case_train_epoch(model, 
                            d_model, 
                            BATCH_SIZE, 
                            train_gen, 
                            tar_gen,
                            weighted_crossentropy,
                            binary_crossentropy,
                            optimizer1,
                            optimizer2,
                            accuracy,
                            binary_accuracy
                            )

    # write the results in tf.summary
    with train_summary_writer.as_default():
        tf.summary.scalar('train segmentation loss', segmentation_loss, step=epoch)
        tf.summary.scalar('train segmentation accuracy', segmentation_accuracy, step=epoch)
        tf.summary.scalar('train domain loss', domain_loss, step=epoch)
        tf.summary.scalar('train domain accuracy', domain_accuracy, step=epoch)

    print('Train: Epoch %s: Seg Loss: %.4f, Seg Acc: %.4f, Dom Loss: %.4f, Dom Acc: %.4f' % 
        (epoch, segmentation_loss, segmentation_accuracy, domain_loss, domain_accuracy))
    
    if (epoch + 1) % 5 == 0:
            (segmentatin_loss,
            segmentation_accuracy,
            domain_loss,
            domain_accuracy) = ignore_case_validation_epoch(model, 
                                d_model, 
                                BATCH_SIZE, 
                                val_gen, 
                                tar_gen,
                                weighted_crossentropy,
                                binary_crossentropy,
                                accuracy,
                                binary_accuracy
                                )
            with test_summary_writer.as_default():
                tf.summary.scalar('test segmentation loss', segmentatin_loss, step=epoch)
                tf.summary.scalar('test segmentation accuracy', segmentation_accuracy, step=epoch)
                tf.summary.scalar('test domain loss', domain_loss, step=epoch)
                tf.summary.scalar('test domain accuracy', domain_accuracy, step=epoch)

            print('Test: Epoch %s: Seg Loss: %.4f, Seg Acc: %.4f, Dom Loss: %.4f, Dom Acc: %.4f' % 
                    (epoch, segmentation_loss, segmentation_accuracy, domain_loss, domain_accuracy))
            
            model.save_weights(f'/content/drive/MyDrive/weights/DAUNet8/{epoch}')

# Load best weights

In [ ]:
model.load_weights("/content/drive/MyDrive/weights/DAUNet8/74")

# Calculate metrics

In [ ]:
plot_confusion_matrix_for_ignore_case(model, train_gen)

In [ ]:
plot_confusion_matrix_for_ignore_case(model, val_gen)

In [ ]:
plot_confusion_matrix_for_ignore_case(model, test_gen)

In [ ]:
plot_confusion_matrix_for_ignore_case(model, tar_gen)

In [ ]:
meanf1_iou_for_ignore_case(model, test_gen, ignore=4)
meanf1_iou_for_ignore_case(model, tar_gen, ignore=4)

In [ ]:
plot_feature_space(model, files, tar_files, minimum, 
                                maximum, normalize=True, gan=False)